In [1]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import calendar
import numpy as np
import time
import pandas as pd
import dask.dataframe as dd
import datetime
import matplotlib.pyplot as plt
import numpy as np

# 1. Overview of the Dataset

## 1.1. trips data

In [2]:
df_trips = pd.read_csv('/Users/ziyingwang/Documents/UCD/Summer/tmp/data/rt_trips_DB_2018.txt', sep=';',error_bad_lines=False)

In [3]:
# print the number of rows and features in the trips dataset
num_rows = df_trips.shape[0]
features = df_trips.shape[1]
print(f"The dataset has {num_rows} rows with {features} features.")

The dataset has 2182637 rows with 16 features.


In [4]:
# print the first 5 rows
df_trips.head(5)

,DATASOURCE,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,BASIN,TENDERLOT,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
0,DB,07-FEB-18 00:00:00,6253783,68,68_80,1,87245,84600,87524.0,84600.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2967409,"
1,DB,07-FEB-18 00:00:00,6262138,25B,25B_271,2,30517,26460,32752.0,NaN,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2580260,"
2,DB,07-FEB-18 00:00:00,6254942,45A,45A_70,2,35512,32100,36329.0,32082.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2448968,"
3,DB,07-FEB-18 00:00:00,6259460,25A,25A_273,1,57261,54420,58463.0,54443.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",3094242,"
4,DB,07-FEB-18 00:00:00,6253175,14,14_15,1,85383,81600,84682.0,81608.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2526331,"


In [5]:
# print the feature names in this dataset
df_trips.keys()

Index(['DATASOURCE', 'DAYOFSERVICE', 'TRIPID', 'LINEID', 'ROUTEID',
       'DIRECTION', 'PLANNEDTIME_ARR', 'PLANNEDTIME_DEP', 'ACTUALTIME_ARR',
       'ACTUALTIME_DEP', 'BASIN', 'TENDERLOT', 'SUPPRESSED', 'JUSTIFICATIONID',
       'LASTUPDATE', 'NOTE'],
      dtype='object')

- DATASOURCE: Unique Bus Operator Code
- DAYOFSERVICE: Day of service. AVL data refer to one day of service that could be greater than 24 hours. For example a trip starting at 23:30 of 1st of April and ending at 00:30 of the 2nd of April could have the last AVL data tracked at “24:30 of 1st of April”. To express this “extended time” the interface requires the use of seconds past the beginning of the operation day. In our example the DayOfService is “01/04/2014” and the extended time is “88200” (= 24*60*60 + 30*60)
- TRIPID: Unique trip code
- LINEID: Unique line code
- ROUTEID: Unique route code
- DIRECTION: Route direction: IB = inbound / going / northbound / eastbound OB = outbound / back / southbound / westbound
- PLANNEDTIME_ARR: Planned departure time of the trip, in seconds
- PLANNEDTIME_DEP: Planned arrival time of the trip, in seconds
- ACTUALTIME_ARR: Actual departure time of the trip, in seconds
- ACTUALTIME_DEP: Actual arrival time of the trip, in seconds
- BASIN: Basin code
- TENDERLOT: Tender lot
- SUPPRESSED: The whole trip has been suppressed (0 =achieved, 1 = suppressed)
- JUSTIFICATIONID: Fault code
- LastUpdate: Time of the last record update
- Note: Free note

In [6]:
# print the number of unique values of the dayofservice feature
len(df_trips['DAYOFSERVICE'].unique())

360

In [7]:
# print the unique values of the dayofservice feature
df_trips['DAYOFSERVICE'].unique()

array(['07-FEB-18 00:00:00', '18-FEB-18 00:00:00', '16-MAR-18 00:00:00',
       '27-MAR-18 00:00:00', '11-MAR-18 00:00:00', '28-SEP-18 00:00:00',
       '26-SEP-18 00:00:00', '26-JUN-18 00:00:00', '21-JUN-18 00:00:00',
       '10-OCT-18 00:00:00', '11-OCT-18 00:00:00', '11-APR-18 00:00:00',
       '10-MAR-18 00:00:00', '20-SEP-18 00:00:00', '29-SEP-18 00:00:00',
       '22-JUN-18 00:00:00', '31-DEC-18 00:00:00', '07-JUL-18 00:00:00',
       '08-JUL-18 00:00:00', '24-SEP-18 00:00:00', '02-DEC-18 00:00:00',
       '09-JUL-18 00:00:00', '15-JUN-18 00:00:00', '16-OCT-18 00:00:00',
       '10-APR-18 00:00:00', '04-JUL-18 00:00:00', '12-APR-18 00:00:00',
       '30-DEC-18 00:00:00', '09-APR-18 00:00:00', '16-FEB-18 00:00:00',
       '21-FEB-18 00:00:00', '03-FEB-18 00:00:00', '12-FEB-18 00:00:00',
       '04-FEB-18 00:00:00', '22-MAR-18 00:00:00', '10-JUL-18 00:00:00',
       '20-APR-18 00:00:00', '19-JUN-18 00:00:00', '16-APR-18 00:00:00',
       '08-FEB-18 00:00:00', '28-OCT-18 00:00:00', 

In [8]:
# print the number of unique values of the tripid feature
len(df_trips['TRIPID'].unique())

658964

Repeat three times on average. Is it uniqute to a route? Will explore later

In [9]:
# print the unique values of the tripid feature
df_trips['TRIPID'].unique()

array([6253783, 6262138, 6254942, ..., 6846574, 6762495, 6762008])

In [10]:
# print the number of unique values of the lineid feature
len(df_trips['LINEID'].unique())

130

In [11]:
# print the unique values of the lineid feature
df_trips['LINEID'].unique()

array(['68', '25B', '45A', '25A', '14', '77A', '39', '16', '40D', '27B',
       '142', '83', '130', '15', '46A', '33', '7', '39A', '49', '1',
       '123', '41', '67X', '59', '9', '40', '239', '76', '84', '53',
       '185', '151', '13', '15B', '65B', '29A', '61', '140', '79A', '38A',
       '31', '33B', '69', '44', '42', '67', '184', '238', '145', '17A',
       '32', '27A', '17', '27X', '18', '122', '54A', '66', '150', '56A',
       '37', '27', '15A', '65', '11', '47', '79', '83A', '63', '4', '120',
       '41C', '70', '84A', '220', '39X', '32X', '68A', '84X', '38', '102',
       '270', '51X', '33X', '75', '26', '66A', '31A', '111', '14C', '114',
       '76A', '44B', '161', '7A', '43', '25', '104', '33A', '16C', '42D',
       '31B', '66X', '31D', '33D', '41B', '40B', '7D', '46E', '38D',
       '118', '51D', '15D', '41A', '25D', '66B', '38B', '236', '7B',
       '41X', '69X', '68X', '25X', '40E', '70D', '116', '77X', '16D',
       '33E', '41D'], dtype=object)

In [12]:
# print the number of unique values of the routeid feature
len(df_trips['ROUTEID'].unique())

588

In [13]:
# print the unique values of the routeid feature
df_trips['ROUTEID'].unique()

array(['68_80', '25B_271', '45A_70', '25A_273', '14_15', '77A_28',
       '39_21', '16_20', '40D_102', '27B_34', '142_9', '83_22', '130_10',
       '15_16', '46A_74', '33_44', '7_70', '39A_40', '39A_43', '49_28',
       '1_40', '123_34', '41_7', '67X_38', '59_10', '9_5', '40_27',
       '239_28', '76_37', '130_11', '84_27', '53_20', '185_55', '41_3',
       '40D_103', '151_15', '13_68', '15B_60', '65B_66', '29A_15',
       '33_70', '61_106', '140_19', '14_16', '7_66', '79A_27', '38A_32',
       '46A_67', '31_18', '33B_58', '15B_61', '69_44', '44_33', '42_42',
       '44_36', '67_6', '184_29', '13_67', '238_15', '145_102', '17A_12',
       '83_19', '45A_64', '32_57', '27A_4', '185_54', '17_16', '27X_43',
       '18_3', '122_16', '25A_270', '122_17', '54A_12', '66_11', '7_72',
       '150_8', '56A_31', '37_15', '27_17', '27A_5', '42_44', '15A_84',
       '65_77', '11_43', '123_36', '145_105', '9_7', '150_9', '47_139',
       '79_11', '79A_28', '83A_20', '63_25', '33B_56', '27B_23', '4_10

each line has multiple routes

In [14]:
# print the unique values of the direction feature
df_trips['DIRECTION'].unique()

array([1, 2])

In [15]:
# print the unique values of the basin feature
df_trips['BASIN'].unique()

array(['BasDef'], dtype=object)

In [16]:
# print the unique values of the tenderlot feature
df_trips['TENDERLOT'].unique()

array([nan])

In [17]:
# print the unique values of the suppressed feature
df_trips['SUPPRESSED'].unique()

array([nan,  0.])

In [18]:
# print the value counts of the suppressed feature values
df_trips['SUPPRESSED'].value_counts()

0.0    4333
Name: SUPPRESSED, dtype: int64

In [19]:
# print the unique values of the justificationid feature
df_trips['JUSTIFICATIONID'].unique()

array([    nan, 194642., 205589., ..., 231826., 231813., 231765.])

In [20]:
# print the unique values of the justificationid feature
df_trips['JUSTIFICATIONID'].unique()

array([    nan, 194642., 205589., ..., 231826., 231813., 231765.])

## 1.2. Vehicles Data

In [21]:
df_vehicles = pd.read_csv('/Users/ziyingwang/Documents/UCD/Summer/tmp/data/rt_vehicles_DB_2018.txt', sep=';',error_bad_lines=False)
# print the number of rows and features in the vehicles dataset
num_rows = df_vehicles.shape[0]
features = df_vehicles.shape[1]
print(f"The dataset has {num_rows} rows with {features} features.")

The dataset has 272622 rows with 7 features.


In [22]:
# print the first 5 rows
df_vehicles.head(5)

,DATASOURCE,DAYOFSERVICE,VEHICLEID,DISTANCE,MINUTES,LASTUPDATE,NOTE
0,DB,23-NOV-18 00:00:00,3303848,286166,58849,04-DEC-18 08:03:09,NaN
1,DB,23-NOV-18 00:00:00,3303847,259545,56828,04-DEC-18 08:03:09,NaN
2,DB,28-FEB-18 00:00:00,2868329,103096,40967,08-MAR-18 10:35:59,NaN
3,DB,28-FEB-18 00:00:00,2868330,147277,43599,08-MAR-18 10:35:59,NaN
4,DB,28-FEB-18 00:00:00,2868331,224682,40447,08-MAR-18 10:35:59,NaN


In [23]:
# Find and print the number of duplicate rows
df = df_vehicles
num_duplicate_rows = df.duplicated().sum()
print(f"There are {num_duplicate_rows} duplicated rows in this dataset (excluding the first row).")
num_duplicate_rows_inclusive = df[df.duplicated(keep=False)].shape[0]
print(f"There are {num_duplicate_rows_inclusive} duplicated rows in this dataset (including row that is duplicated).")

There are 0 duplicated rows in this dataset (excluding the first row).
There are 0 duplicated rows in this dataset (including row that is duplicated).


## 1.3. leavetimes data

In [25]:
df_leavetimes = pd.read_csv('/Users/ziyingwang/Documents/UCD/Summer/tmp/data/rt_leavetimes_DB_2018.txt', sep=';',error_bad_lines=False)

In [26]:
num_rows = df_leavetimes.shape[0]
features = df_leavetimes.shape[1]
print(f"The dataset has {num_rows} rows with {features} features.")

The dataset has 116949113 rows with 18 features.


In [27]:
df_leavetimes.head(5)

,DATASOURCE,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,PASSENGERS,PASSENGERSIN,PASSENGERSOUT,DISTANCE,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
0,DB,01-JAN-18 00:00:00,5972116,12,119,48030,48030,48012,48012,2693211,NaN,NaN,NaN,NaN,NaN,NaN,08-JAN-18 17:21:10,NaN
1,DB,01-JAN-18 00:00:00,5966674,12,119,54001,54001,54023,54023,2693267,NaN,NaN,NaN,NaN,NaN,NaN,08-JAN-18 17:21:10,NaN
2,DB,01-JAN-18 00:00:00,5959105,12,119,60001,60001,59955,59955,2693263,NaN,NaN,NaN,NaN,NaN,NaN,08-JAN-18 17:21:10,NaN
3,DB,01-JAN-18 00:00:00,5966888,12,119,58801,58801,58771,58771,2693284,NaN,NaN,NaN,NaN,NaN,NaN,08-JAN-18 17:21:10,NaN
4,DB,01-JAN-18 00:00:00,5965960,12,119,56401,56401,56309,56323,2693209,NaN,NaN,NaN,NaN,NaN,NaN,08-JAN-18 17:21:10,NaN


In [28]:
df_leavetimes.tail(5)

,DATASOURCE,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,PASSENGERS,PASSENGERSIN,PASSENGERSOUT,DISTANCE,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
116949108,DB,31-DEC-18 00:00:00,8588153,78,4383,28605,28605,28998,29013,3265721,NaN,NaN,NaN,NaN,NaN,NaN,16-JAN-19 18:27:21,NaN
116949109,DB,31-DEC-18 00:00:00,8587459,78,4383,22695,22695,23247,23247,3265687,NaN,NaN,NaN,NaN,NaN,NaN,16-JAN-19 18:27:21,NaN
116949110,DB,31-DEC-18 00:00:00,8586183,78,4383,51481,51481,52237,52283,2693229,NaN,NaN,NaN,NaN,NaN,NaN,16-JAN-19 18:27:21,NaN
116949111,DB,31-DEC-18 00:00:00,8589374,23,7053,53659,53659,53525,53525,3265669,NaN,NaN,NaN,NaN,NaN,NaN,16-JAN-19 18:27:21,NaN
116949112,DB,31-DEC-18 00:00:00,8589372,24,2088,46383,46383,46315,46325,3265669,NaN,NaN,NaN,NaN,NaN,NaN,16-JAN-19 18:27:21,NaN


In [29]:
# print the features of the dataset
df_leavetimes.keys()

Index(['DATASOURCE', 'DAYOFSERVICE', 'TRIPID', 'PROGRNUMBER', 'STOPPOINTID',
       'PLANNEDTIME_ARR', 'PLANNEDTIME_DEP', 'ACTUALTIME_ARR',
       'ACTUALTIME_DEP', 'VEHICLEID', 'PASSENGERS', 'PASSENGERSIN',
       'PASSENGERSOUT', 'DISTANCE', 'SUPPRESSED', 'JUSTIFICATIONID',
       'LASTUPDATE', 'NOTE'],
      dtype='object')

- DAYOFSERVICE: date
- TRIPID: unique trip code - there will be multiple trips in one route so it is used to differentiate each trip
- PROGRNUMBER: sequential position of the stop point in the trip - shows how far a specific route has gone
- STOPPOINTID: unique stop point code - might be number on the stop?
- PLANNEDTIME_ARR: planned arrival time at the stop point (seconds)
- PLANNEDTIME_DEP: planned departure time from the stop point (seconds)
- ACTUALTIME_ARR: actual arrival time at the stop point (seconds)
- ACTUALTIME_DEP: actual departure time from the stop point (seconds)
- VEHICLEID: unique vehicle code arriving at this stop point
- PASSENGERS: num of passengers on board
- PASSENGERSIN: num of boarded passengers
- PASSENGERSOUT: num of descended passengers
- DISTANCE: distance measured from the beginning of the trip
- SUPPRESSED: when the trip is partially suppressed it says that the previous link is suppressed (0 = achieved, 1 = suppressed)
- JUSTIFICATIONID: fault code

In [30]:
# print the unique values of the datasource feature
df_leavetimes['DATASOURCE'].unique()

array(['DB'], dtype=object)

In [31]:
# print the number of unique values for dayofservice
len(df_leavetimes['DAYOFSERVICE'].unique())

360

In [32]:
# print the number of unique values for tripid
len(df_leavetimes['TRIPID'].unique())

658961

Compared to trip datasets, 3 trips are missing?

In [33]:
# print the number of unique values for PROGRNUMBER
len(df_leavetimes['PROGRNUMBER'].unique())

109

In [34]:
# print the number of unique values for distance
len(df_leavetimes['DISTANCE'].unique())

1

In [35]:
# print the value counts of the distance feature values
df_leavetimes['DISTANCE'].value_counts()

Series([], Name: DISTANCE, dtype: int64)

If we want to do a full route estimation first, stop% variable cannot be calculated based on distance. We might calculate % based on travel time?

# 2. Routes and Stops Analysis

In [52]:
# make a new df of entries with only that trip id
df_single_route = df_trips[df_trips['TRIPID'] == 6253783]
df_single_route

,DATASOURCE,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,BASIN,TENDERLOT,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
0,DB,07-FEB-18 00:00:00,6253783,68,68_80,1,87245,84600,87524.0,84600.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2967409,"
123487,DB,15-FEB-18 00:00:00,6253783,68,68_80,1,87245,84600,87703.0,84665.0,BasDef,NaN,NaN,NaN,28-FEB-18 13:51:50,",2967409,"
246486,DB,16-FEB-18 00:00:00,6253783,68,68_80,1,87245,84600,88134.0,84604.0,BasDef,NaN,NaN,NaN,26-FEB-18 12:19:30,",2967409,"
1057217,DB,08-FEB-18 00:00:00,6253783,68,68_80,1,87245,84600,87898.0,84616.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:20:01,",2967409,"
1135435,DB,12-FEB-18 00:00:00,6253783,68,68_80,1,87245,84600,87480.0,84554.0,BasDef,NaN,NaN,NaN,28-FEB-18 13:18:29,",2967409,"
1192222,DB,05-FEB-18 00:00:00,6253783,68,68_80,1,87245,84600,87043.0,84608.0,BasDef,NaN,NaN,NaN,28-FEB-18 11:29:31,",2967409,"
1310634,DB,09-FEB-18 00:00:00,6253783,68,68_80,1,87245,84600,87788.0,84609.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:32:51,",2967409,"
1576426,DB,13-FEB-18 00:00:00,6253783,68,68_80,1,87245,84600,87184.0,NaN,BasDef,NaN,NaN,NaN,28-FEB-18 13:29:11,",2967409,"
1615621,DB,06-FEB-18 00:00:00,6253783,68,68_80,1,87245,84600,87448.0,84548.0,BasDef,NaN,NaN,NaN,28-FEB-18 11:48:21,",2967409,"
1652270,DB,14-FEB-18 00:00:00,6253783,68,68_80,1,87245,84600,87688.0,84599.0,BasDef,NaN,NaN,NaN,15-MAR-18 12:47:59,",2967409,"


In [53]:
# only keep one route
df_single_route = df_single_route[df_single_route['DAYOFSERVICE'] == '07-FEB-18 00:00:00']
df_single_route

,DATASOURCE,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,BASIN,TENDERLOT,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
0,DB,07-FEB-18 00:00:00,6253783,68,68_80,1,87245,84600,87524.0,84600.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2967409,"


It seems that one tripid serves one routeid but is used on different days.  Pick one day for further analysis.

In [49]:
# make a new df of entries with only that trip id
df_single_route_leavetimes = df_leavetimes[df_leavetimes['TRIPID'] == 6253783]
df_single_route_leavetimes

,DATASOURCE,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,PASSENGERS,PASSENGERSIN,PASSENGERSOUT,DISTANCE,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
12066788,DB,16-FEB-18 00:00:00,6253783,40,4379,86107,86107,86752,86752,2693290,NaN,NaN,NaN,NaN,NaN,NaN,26-FEB-18 12:19:30,NaN
12066790,DB,16-FEB-18 00:00:00,6253783,41,1968,86198,86198,86859,86859,2693290,NaN,NaN,NaN,NaN,NaN,NaN,26-FEB-18 12:19:30,NaN
12066792,DB,16-FEB-18 00:00:00,6253783,42,1969,86215,86215,86868,86868,2693290,NaN,NaN,NaN,NaN,NaN,NaN,26-FEB-18 12:19:30,NaN
12066795,DB,16-FEB-18 00:00:00,6253783,44,1971,86277,86277,86951,86961,2693290,NaN,NaN,NaN,NaN,NaN,NaN,26-FEB-18 12:19:30,NaN
12066797,DB,16-FEB-18 00:00:00,6253783,45,2114,86379,86379,87122,87130,2693290,NaN,NaN,NaN,NaN,NaN,NaN,26-FEB-18 12:19:30,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19569148,DB,07-FEB-18 00:00:00,6253783,31,1952,85789,85789,86004,86019,2534790,NaN,NaN,NaN,NaN,NaN,NaN,28-FEB-18 12:05:11,NaN
19569149,DB,07-FEB-18 00:00:00,6253783,41,1968,86198,86198,86451,86463,2534790,NaN,NaN,NaN,NaN,NaN,NaN,28-FEB-18 12:05:11,NaN
19569150,DB,07-FEB-18 00:00:00,6253783,51,3378,86602,86602,86898,86910,2534790,NaN,NaN,NaN,NaN,NaN,NaN,28-FEB-18 12:05:11,NaN
19569151,DB,07-FEB-18 00:00:00,6253783,60,3386,86900,86900,87209,87209,2534790,NaN,NaN,NaN,NaN,NaN,NaN,28-FEB-18 12:05:11,NaN


In [51]:
df_single_route_leavetimes = df_single_route_leavetimes[df_single_route_leavetimes['DAYOFSERVICE'] == '07-FEB-18 00:00:00']
df_single_route_leavetimes

,DATASOURCE,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,PASSENGERS,PASSENGERSIN,PASSENGERSOUT,DISTANCE,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
13382799,DB,07-FEB-18 00:00:00,6253783,1,4495,84600,84600,84600,84600,2534790,NaN,NaN,NaN,NaN,NaN,NaN,28-FEB-18 12:05:11,NaN
13382800,DB,07-FEB-18 00:00:00,6253783,4,1278,84713,84713,84827,84838,2534790,NaN,NaN,NaN,NaN,NaN,NaN,28-FEB-18 12:05:11,NaN
13382801,DB,07-FEB-18 00:00:00,6253783,5,1282,84827,84827,84917,84928,2534790,NaN,NaN,NaN,NaN,NaN,NaN,28-FEB-18 12:05:11,NaN
13382802,DB,07-FEB-18 00:00:00,6253783,6,4456,84882,84882,84970,84970,2534790,NaN,NaN,NaN,NaN,NaN,NaN,28-FEB-18 12:05:11,NaN
13382803,DB,07-FEB-18 00:00:00,6253783,7,1284,84921,84921,84991,84991,2534790,NaN,NaN,NaN,NaN,NaN,NaN,28-FEB-18 12:05:11,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19569148,DB,07-FEB-18 00:00:00,6253783,31,1952,85789,85789,86004,86019,2534790,NaN,NaN,NaN,NaN,NaN,NaN,28-FEB-18 12:05:11,NaN
19569149,DB,07-FEB-18 00:00:00,6253783,41,1968,86198,86198,86451,86463,2534790,NaN,NaN,NaN,NaN,NaN,NaN,28-FEB-18 12:05:11,NaN
19569150,DB,07-FEB-18 00:00:00,6253783,51,3378,86602,86602,86898,86910,2534790,NaN,NaN,NaN,NaN,NaN,NaN,28-FEB-18 12:05:11,NaN
19569151,DB,07-FEB-18 00:00:00,6253783,60,3386,86900,86900,87209,87209,2534790,NaN,NaN,NaN,NaN,NaN,NaN,28-FEB-18 12:05:11,NaN


In [54]:
df_single_route_stop['STOPPOINTID'].unique()

array([4495, 1278, 1282, 4456, 1284, 1285, 1286, 1288, 1360, 1362, 1363,
       1364, 1365, 1366, 1367, 1369, 1370, 1372, 1373, 1374, 1946, 1947,
       1948, 1949, 1950, 1951, 1953, 1954, 1955, 1956, 1957, 1958, 1959,
       1960, 4379, 1969, 1970, 1971, 2114, 2115, 6011, 2133, 6244, 2134,
       3379, 4669, 2165, 2166, 2167, 3383, 3384, 3385, 3387, 4559, 3389,
       3390, 6119, 3392, 3393, 3396, 3397, 3394, 3399, 1289, 1371, 1952,
       1968, 3378, 3386, 3398])

The route has 71 stops. This number is quite large. May be this route can be selected as single route estimation model in later machine learning training.